# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f35683e5d30>
├── 'a' --> tensor([[-1.0803,  0.3495,  1.9116],
│                   [-0.1099, -1.4414, -0.5323]])
└── 'x' --> <FastTreeValue 0x7f35683e5970>
    └── 'c' --> tensor([[ 0.4257,  0.5443, -1.5365,  1.3197],
                        [ 2.1281, -1.5547, -0.1908,  0.2573],
                        [-0.1966, -1.2311,  0.3959, -1.7758]])

In [4]:
t.a

tensor([[-1.0803,  0.3495,  1.9116],
        [-0.1099, -1.4414, -0.5323]])

In [5]:
%timeit t.a

85.8 ns ± 0.0801 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f35683e5d30>
├── 'a' --> tensor([[ 1.3146, -0.9747,  1.4095],
│                   [-0.6825,  0.1936,  0.4942]])
└── 'x' --> <FastTreeValue 0x7f35683e5970>
    └── 'c' --> tensor([[ 0.4257,  0.5443, -1.5365,  1.3197],
                        [ 2.1281, -1.5547, -0.1908,  0.2573],
                        [-0.1966, -1.2311,  0.3959, -1.7758]])

In [7]:
%timeit t.a = new_value

86.4 ns ± 0.385 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.0803,  0.3495,  1.9116],
               [-0.1099, -1.4414, -0.5323]]),
    x: Batch(
           c: tensor([[ 0.4257,  0.5443, -1.5365,  1.3197],
                      [ 2.1281, -1.5547, -0.1908,  0.2573],
                      [-0.1966, -1.2311,  0.3959, -1.7758]]),
       ),
)

In [10]:
b.a

tensor([[-1.0803,  0.3495,  1.9116],
        [-0.1099, -1.4414, -0.5323]])

In [11]:
%timeit b.a

70.4 ns ± 0.532 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.3110, -0.7606, -1.3623],
               [ 0.4021, -1.3403, -0.3975]]),
    x: Batch(
           c: tensor([[ 0.4257,  0.5443, -1.5365,  1.3197],
                      [ 2.1281, -1.5547, -0.1908,  0.2573],
                      [-0.1966, -1.2311,  0.3959, -1.7758]]),
       ),
)

In [13]:
%timeit b.a = new_value

583 ns ± 4.19 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

965 ns ± 1.71 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.2 µs ± 63.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

166 µs ± 1.94 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

165 µs ± 681 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f353c94d730>
├── 'a' --> tensor([[[-1.0803,  0.3495,  1.9116],
│                    [-0.1099, -1.4414, -0.5323]],
│           
│                   [[-1.0803,  0.3495,  1.9116],
│                    [-0.1099, -1.4414, -0.5323]],
│           
│                   [[-1.0803,  0.3495,  1.9116],
│                    [-0.1099, -1.4414, -0.5323]],
│           
│                   [[-1.0803,  0.3495,  1.9116],
│                    [-0.1099, -1.4414, -0.5323]],
│           
│                   [[-1.0803,  0.3495,  1.9116],
│                    [-0.1099, -1.4414, -0.5323]],
│           
│                   [[-1.0803,  0.3495,  1.9116],
│                    [-0.1099, -1.4414, -0.5323]],
│           
│                   [[-1.0803,  0.3495,  1.9116],
│                    [-0.1099, -1.4414, -0.5323]],
│           
│                   [[-1.0803,  0.3495,  1.9116],
│                    [-0.1099, -1.4414, -0.5323]]])
└── 'x' --> <FastTreeValue 0x7f35683f0eb0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

37.8 µs ± 258 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f34bb0e9fd0>
├── 'a' --> tensor([[-1.0803,  0.3495,  1.9116],
│                   [-0.1099, -1.4414, -0.5323],
│                   [-1.0803,  0.3495,  1.9116],
│                   [-0.1099, -1.4414, -0.5323],
│                   [-1.0803,  0.3495,  1.9116],
│                   [-0.1099, -1.4414, -0.5323],
│                   [-1.0803,  0.3495,  1.9116],
│                   [-0.1099, -1.4414, -0.5323],
│                   [-1.0803,  0.3495,  1.9116],
│                   [-0.1099, -1.4414, -0.5323],
│                   [-1.0803,  0.3495,  1.9116],
│                   [-0.1099, -1.4414, -0.5323],
│                   [-1.0803,  0.3495,  1.9116],
│                   [-0.1099, -1.4414, -0.5323],
│                   [-1.0803,  0.3495,  1.9116],
│                   [-0.1099, -1.4414, -0.5323]])
└── 'x' --> <FastTreeValue 0x7f34bb0dabb0>
    └── 'c' --> tensor([[ 0.4257,  0.5443, -1.5365,  1.3197],
                        [ 2.1281, -1.5547, -0.1908,  0.2573],
                 

In [23]:
%timeit t_cat(trees)

35.4 µs ± 115 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

66.7 µs ± 2.68 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-1.0803,  0.3495,  1.9116],
                [-0.1099, -1.4414, -0.5323]],
       
               [[-1.0803,  0.3495,  1.9116],
                [-0.1099, -1.4414, -0.5323]],
       
               [[-1.0803,  0.3495,  1.9116],
                [-0.1099, -1.4414, -0.5323]],
       
               [[-1.0803,  0.3495,  1.9116],
                [-0.1099, -1.4414, -0.5323]],
       
               [[-1.0803,  0.3495,  1.9116],
                [-0.1099, -1.4414, -0.5323]],
       
               [[-1.0803,  0.3495,  1.9116],
                [-0.1099, -1.4414, -0.5323]],
       
               [[-1.0803,  0.3495,  1.9116],
                [-0.1099, -1.4414, -0.5323]],
       
               [[-1.0803,  0.3495,  1.9116],
                [-0.1099, -1.4414, -0.5323]]]),
    x: Batch(
           c: tensor([[[ 0.4257,  0.5443, -1.5365,  1.3197],
                       [ 2.1281, -1.5547, -0.1908,  0.2573],
                       [-0.1966, -1.2311,  0.3959, -1.7758]],
         

In [26]:
%timeit Batch.stack(batches)

89.7 µs ± 276 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-1.0803,  0.3495,  1.9116],
               [-0.1099, -1.4414, -0.5323],
               [-1.0803,  0.3495,  1.9116],
               [-0.1099, -1.4414, -0.5323],
               [-1.0803,  0.3495,  1.9116],
               [-0.1099, -1.4414, -0.5323],
               [-1.0803,  0.3495,  1.9116],
               [-0.1099, -1.4414, -0.5323],
               [-1.0803,  0.3495,  1.9116],
               [-0.1099, -1.4414, -0.5323],
               [-1.0803,  0.3495,  1.9116],
               [-0.1099, -1.4414, -0.5323],
               [-1.0803,  0.3495,  1.9116],
               [-0.1099, -1.4414, -0.5323],
               [-1.0803,  0.3495,  1.9116],
               [-0.1099, -1.4414, -0.5323]]),
    x: Batch(
           c: tensor([[ 0.4257,  0.5443, -1.5365,  1.3197],
                      [ 2.1281, -1.5547, -0.1908,  0.2573],
                      [-0.1966, -1.2311,  0.3959, -1.7758],
                      [ 0.4257,  0.5443, -1.5365,  1.3197],
                      [ 2.1281, -

In [28]:
%timeit Batch.cat(batches)

165 µs ± 417 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

371 µs ± 3.37 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
